# Nutrition Tracking

1. Program allows users to enter the name of the food and brand owner and get nutrients on the food.
2. The food and brand name must be written exactly as entered in food and branded_food Excel workbooks.
3. In order for the program to work, the user needs to download the latest zipped folder as a CSV file from the Branded Data Type on the FoodData Central Website 
https://fdc.nal.usda.gov/download-datasets.html
4. When running a report, some foods will not record any nutrients or might record very few nutrients. The lack of nutrients is due to the brand company reporting few to no nutrients. No nutrients does not mean a food does not have nutrients.
5. Food's nutrients are recorded per 100g of food, so not the exact same as what label says.

# Load the modules

In [1]:
import pandas as pd

# Load the Files

In [2]:
df_food = pd.read_csv('food.csv')
df_food["description"] = df_food["description"].astype(str)

In [3]:
df_branded_food = pd.read_csv('branded_food.csv', low_memory=False)

In [4]:
df_food_nutrients = pd.read_csv('food_nutrient.csv')

In [5]:
df_nutrient = pd.read_csv("nutrient.csv")

# Record Nutrients Consumed

In [6]:
nutrients = df_nutrient["name"].tolist()
dfmain_data = pd.DataFrame(columns=nutrients)

more = "yes"
    
while more != 'no':   
    # asks user for description of food and converts it to upper case
    user_food_description = input("Enter the exact description of the food: ")
    user_food_description = user_food_description.upper()
    print("Description: " + user_food_description)
    print()

    # splits the food description provided by user into a list and converts all food descriptions to list
    list_user_food_description = user_food_description.split()
    list_all_food_descriptions = df_food["description"].tolist()

    '''loops through list description provided by user and compares it with the list of all food descriptions
    if a word provided by the user is in the list of all food descriptions, append to new list.
    get all columns of foods dataframe where description is in the list of descriptions 
    assigns the list of descriptions to the list of all food descriptions and 
    repeats the process, until all words from the user description have been looked at'''
    list_descriptions = []
    for i in range(0, len(list_user_food_description)):
        for j in range(0, len(list_all_food_descriptions)):
            if list_user_food_description[i] in list_all_food_descriptions[j]:
                list_descriptions.append(list_all_food_descriptions[j])
        df_select_foods = df_food[df_food["description"].isin(list_descriptions)]
        list_all_food_descriptions = list_descriptions
        list_descriptions = []

    '''converts ids of the dataframe select foods to a list.
    Checks the dateframe of branded food and gets the rows where fdc_id is in the list of fdc_ids.
    Creates a new dataframe.'''
    list_fdc_ids = df_select_foods["fdc_id"].tolist()
    df_select_branded_food = df_branded_food[df_branded_food["fdc_id"].isin(list_fdc_ids)]

    # converts the brand owners of the select branded food to a list.
    list_brand_owners = df_select_branded_food["brand_owner"].tolist()

    # get unique brand owners and sort the list
    unique_list = []
    for x in list_brand_owners:
        if x not in unique_list:
            unique_list.append(x)
    unique_list = sorted(unique_list)

    # print out numbered list of unique brands   
    for i in range(0, len(unique_list)):
        print(str(i+1) + ") " + unique_list[i])
    print()

    # ask user which number the brand owner of the food is
    number = int(input("Which number is your brand?: "))
    brand = unique_list[number-1]

    # creates new dataframe of the brand provided by user
    df_selected_brand = df_select_branded_food[df_select_branded_food["brand_owner"] == brand]

    list_fdc_ids = df_selected_brand["fdc_id"].tolist()

    df_selected_foods = df_food[df_food["fdc_id"].isin(list_fdc_ids)]

    df_food_unique = df_selected_foods.iloc[-1]

    fdc_id = df_food_unique["fdc_id"]

    # Get all the nutrients that are equal to the fdcId
    df_food_nutrients_id = df_selected_foods[df_selected_foods["fdc_id"] == fdc_id]

    df_nutrients_id = df_food_nutrients[df_food_nutrients["fdc_id"] == fdc_id]
    df_nutrient2 = df_nutrients_id[["nutrient_id","amount"]]

    nutrient_ids = df_nutrient2["nutrient_id"].tolist()

    df_nutrients = df_nutrient[df_nutrient["id"].isin(nutrient_ids)]
    dfmain = df_nutrients.merge(df_nutrient2, left_on=['id'], right_on=['nutrient_id'], how = 'outer')
    dfmain = dfmain.drop(columns=["nutrient_id"])

    list_nutrient_names = dfmain["name"].tolist()
    list_nutrient_units = dfmain["unit_name"].tolist()
    list_nutrient_amounts = dfmain["amount"].tolist()

    df_selected_brand = df_selected_brand[df_selected_brand["fdc_id"] == fdc_id]

    serving_size = df_selected_brand["serving_size"].tolist()
    serving_size_unit = df_selected_brand["serving_size_unit"].tolist()
    household_serving = df_selected_brand["household_serving_fulltext"].tolist()

    print()
    print("Serving size is " + str(serving_size[0]) + serving_size_unit[0])
    print("equivalent to " + str(household_serving[0]))
    print()
    servings = input("How many servings did you consume?: ")

    # convert 100 grams to 1 serving
    serving_size_consumed = serving_size[0] * float(servings)
    serving_size_nutrients = 100
    serving_size_nutrients_consumed = serving_size_nutrients / serving_size_consumed

    for i in range(0, len(list_nutrient_amounts)):
        list_nutrient_amounts[i] /= serving_size_nutrients_consumed

    list_nutrient_amounts = [ '%.2f' % elem for elem in list_nutrient_amounts ]

    print()
    print("Nutrients:")
    for i in range(0, len(list_nutrient_names)):
        print(list_nutrient_names[i])
        print(str(list_nutrient_amounts[i]) + " " + str(list_nutrient_units[i]))

    print()
    ingredients = df_selected_brand["ingredients"].iloc[0]
    print("Ingredients: ")
    print(ingredients)

    nutrients = df_nutrient["name"].tolist()
    dfmain1 = pd.DataFrame(columns=nutrients)

    # create the dataframe to copy
    amounts_totals = []
    for i in range(0, len(nutrients)):
        total = 0.0
        for j in range(0, len(list_nutrient_names)):
            if list_nutrient_names[j] == nutrients[i]:
                total = list_nutrient_amounts[j]
        amounts_totals.append(total)

    df = pd.DataFrame(columns=nutrients)
    df.loc[0] = amounts_totals

    # concat the dataframe and append with the main dataframe
    dfmain1 = pd.concat([dfmain1, df])
    dfmain_data = dfmain_data.append(dfmain1)
    
    print()
    more = input("Do you have more food to add? (yes or no): ")

dfmain_data.to_excel("output.xlsx", index=False)

Enter the exact description of the food: nature valley protein peanut butter dark chocolate
Description: NATURE VALLEY PROTEIN PEANUT BUTTER DARK CHOCOLATE

1) General Mills, Inc.

Which number is your brand?: 1

Serving size is 40.0g
equivalent to nan

How many servings did you consume?: 1

Nutrients:
Protein
10.00 G
Total lipid (fat)
12.00 G
Carbohydrate, by difference
14.00 G
Energy
190.00 KCAL
Fiber, total dietary
5.00 G
Calcium, Ca
20.00 MG
Iron, Fe
1.08 MG
Sodium, Na
180.00 MG
Cholesterol
0.00 MG
Fatty acids, total trans
0.00 G
Fatty acids, total saturated
3.50 G
Fatty acids, total monounsaturated
7.00 G
Fatty acids, total polyunsaturated
1.00 G
Sugars, total including NLEA
6.00 G

Ingredients: 
ROASTED PEANUTS, SOY PROTEIN ISOLATE, CHICORY ROOT EXTRACT, SEMISWEET CHOCOLATE CHIPS (SUGAR, CHOCOLATE LIQUOR, COCOA BUTTER, SOY LECITHIN, NATURAL FLAVOR) VEGETABLE OILS (PALM KERNEL, PALM, CANOLA, PEANUT), SUGAR. CORN SYRUP, WHEY PROTEIN CONCENTRATE, FRUCTOSE, PEANUT BUTTER (PEANUTS, SA

C:\Users\kyles\AppData\Local\Temp\ipykernel_15488\935088884.py:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfmain_data = dfmain_data.append(dfmain1)



Do you have more food to add? (yes or no): yes
Enter the exact description of the food: alaskan pink salmon
Description: ALASKAN PINK SALMON

1) American Tuna
2) Associated Wholesale Grocers, Inc.
3) BJ's Wholesale Club / Corporate Brands
4) CROWN PRINCE
5) Crown Prince, Inc.
6) DOUBLE "Q"
7) DOUBLE"Q"
8) Ecofish, Inc.
9) HENRY & LISA'S
10) HENRY & LISA'S NATURAL SEAFOOD
11) Iga, Inc.
12) Learning Horizons
13) Meijer, Inc.
14) Motion Analytics Corporation
15) Ocean Beauty Seafoods LLC
16) Peter Pan Seafoods, Inc.
17) Rich Products Corporation
18) Safeway, Inc.
19) StarKist Co.
20) Supervalu, Inc.
21) WEIS QUALITY
22) Wal-Mart Stores, Inc.
23) Weis Markets, Inc.
24) Whole Foods Market, Inc.
25) Yihe Corporation

Which number is your brand?: 22

Serving size is 63.0MG
equivalent to 0.25 cup

How many servings did you consume?: 1.5


C:\Users\kyles\AppData\Local\Temp\ipykernel_15488\935088884.py:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfmain_data = dfmain_data.append(dfmain1)



Nutrients:
Protein
18.00 G
Total lipid (fat)
4.50 G
Carbohydrate, by difference
0.00 G
Energy
120.02 KCAL
Fiber, total dietary
0.00 G
Calcium, Ca
224.91 MG
Iron, Fe
0.54 MG
Potassium, K
329.81 MG
Sodium, Na
344.93 MG
Vitamin A, IU
0.00 IU
Vitamin C, total ascorbic acid
0.00 MG
Cholesterol
52.92 MG
Fatty acids, total trans
0.00 G
Fatty acids, total saturated
0.75 G
Fatty acids, total monounsaturated
1.50 G
Fatty acids, total polyunsaturated
1.50 G
Sugars, total including NLEA
0.00 G

Ingredients: 
PINK SALMON, SALT.

Do you have more food to add? (yes or no): no
